# First steps with Claude throught Bedrock

## The most simple approach

In [2]:
import os
import boto3
from dotenv import load_dotenv

In [3]:
load_dotenv() # load the environment variables

True

In [4]:
profile_name=os.getenv("PROFILE_NAME") #  I currently log in using SSO.

In [5]:
session = boto3.session.Session(profile_name=profile_name, region_name="us-west-2")

In [6]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

user_message = {
    "role": "user",
    "content": [
        {"text": "What's 1+1?"}
    ]
}

response = client.converse(
    modelId = model_id,
    messages = [user_message]
)

In [7]:
response["output"]["message"] 

{'role': 'assistant', 'content': [{'text': '1 + 1 = 2'}]}

## Multi-turn approach

In [8]:
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def add_user_message(messages: list, text: str):
        
    user_message = {
        "role": "user",
        "content": [
            {"text": text}
        ]
    }

    messages.append(user_message)
    

def add_assistant_message(messages: list, text: str):
        
    assistant_message = {
        "role": "assistant",
        "content": [
            {"text": text}
        ]
    }

    messages.append(assistant_message) 

def chat(messages: list):
    response = client.converse(
        modelId = model_id,
        messages = messages
    )

    return response["output"]["message"]["content"][0]["text"]

In [9]:
# started with an empty list of messages where we maintaining the context
messages = []

# initial user question
add_user_message(messages, "What's 1+1?")

# pass the list of messages into chat
answer = chat(messages)

# take the answer from the assistant and added it into the message list  
add_assistant_message(messages, answer)

# followup question
add_user_message(messages, "added 3 more, what's the result?")

# chat again
answer = chat(messages)

answer

'If we add 3 more to the previous result of 2:\n\n2 + 3 = 5'

### Query Processing

In [10]:
messages=[]
client = session.client("bedrock-runtime", region_name="us-west-2")
model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0" # this refer to inference profile ID

def process_query(query: str):
    
    add_user_message(messages, query)
    answer = chat(messages)
    add_assistant_message(messages, answer)
    return answer
                

# Chat Loop

In [18]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("Query: ").strip()
            if query.lower()=='quit':
                break
            assistant_answer = process_query(query)
            print("\nAssistant:", assistant_answer)
        except Exception as e:
            print(f"\nError: {str(e)}")

In [19]:
chat_loop()

Type your queries or 'quit' to exit.


Query:  What's 1+1?



Assistant: 1 + 1 = 2


Query:  add 3 



Assistant: 2 + 3 = 5


Query:  thank you. 



Assistant: You're welcome! Happy to help with the math.


Query:  quit
